In [61]:
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import ultralytics
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
from pathlib import Path
import glob
import optuna
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)
import matplotlib.pyplot as plt

In [62]:
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"Ultralytics 버전: {ultralytics.__version__}")
print(f"CUDA 버전: {torch.version.cuda}")

model = YOLO('yolov8n.pt')
print(f"모델 작업 유형: {model.task}")

PyTorch 버전: 2.0.1+cu117
CUDA 사용 가능: True
Ultralytics 버전: 8.3.0
CUDA 버전: 11.7
모델 작업 유형: detect


In [63]:
# 자동 정제 스크립트 (박스 우선 유지)

def clean_labels(label_dir):
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as f:
            lines = [l for l in f.readlines() if len(l.split()) == 5]
        
        with open(filepath, 'w') as f:
            f.writelines(lines)
        
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/train")
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/val")
clean_labels("C:/Users/user/Downloads/dataset/dataset/labels/test")

In [64]:
# 비어있는 파일이 있는 경우 성능 저하를 야기할 수 있으므로 비어 있는 label 파일과 이미지도 같이 삭제하는 파일

def clean_empty_labels(label_dir, image_dir=None, delete_images=False):
    label_dir = Path(label_dir)
    if image_dir:
        image_dir = Path(image_dir)

    removed_labels = 0
    removed_images = 0

    for label_file in label_dir.glob("*.txt"):
        if label_file.stat().st_size == 0:  # 파일이 비어있는지 확인
            # 라벨 파일 삭제
            label_file.unlink()
            removed_labels += 1

            if delete_images and image_dir:
                # 이미지 파일명 추정 (확장자 jpg, png 등)
                stem = label_file.stem
                for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                    img_file = image_dir / f"{stem}{ext}"
                    if img_file.exists():
                        img_file.unlink()
                        removed_images += 1
                        break

    print(f"삭제된 비어있는 라벨 파일: {removed_labels}개")
    if delete_images:
        print(f"함께 삭제된 이미지 파일: {removed_images}개")

        
# 라벨과 이미지 모두 삭제
clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\train",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\train",
    delete_images=True
)

clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\val",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\val",
    delete_images=True
)

clean_empty_labels(
    label_dir=r"C:\Users\user\Downloads\dataset\dataset\labels\test",
    image_dir=r"C:\Users\user\Downloads\dataset\dataset\images\test",
    delete_images=True
)

삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개
삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개
삭제된 비어있는 라벨 파일: 0개
함께 삭제된 이미지 파일: 0개


In [65]:
# 비어있는 파일이 존재하는지 확인하는 코드

def check_labels(label_path):
    empty_found = False  # 비어있는 파일이 발견되었는지 여부
    for lbl_file in Path(label_path).glob("*.txt"):
        with open(lbl_file) as f:
            lines = f.readlines()
            if not lines:
                print(f"Empty file: {lbl_file}")
                empty_found = True
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Invalid line in {lbl_file}: {line}")
    if not empty_found:
        print(f"[{label_path}] 비어있는 파일이 존재하지 않습니다.")

check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\train")
check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\val")
check_labels(r"C:\Users\user\Downloads\dataset\dataset\labels\test")

[C:\Users\user\Downloads\dataset\dataset\labels\train] 비어있는 파일이 존재하지 않습니다.
[C:\Users\user\Downloads\dataset\dataset\labels\val] 비어있는 파일이 존재하지 않습니다.
[C:\Users\user\Downloads\dataset\dataset\labels\test] 비어있는 파일이 존재하지 않습니다.


In [70]:
# YOLO 학습 과정에서 생성되는 캐시 파일 삭제하는 코드

cache_files = glob.glob(r"C:/Users/user/Downloads/dataset/dataset/labels/*.cache")
for f in cache_files:
    os.remove(f)
print(f"{len(cache_files)}개의 캐시 파일 삭제 완료")


0개의 캐시 파일 삭제 완료


In [81]:
# 클래스별 인스턴스 수를 확인할 수 있는 코드

def count_instance_per_class(name, label_dir):
    class_counts = defaultdict(int)

    for label_file in Path(label_dir).glob("*.txt"):
        with open(label_file) as f:
            for line in f:
                if line.strip():
                    class_id = int(line.split()[0])
                    class_counts[class_id] += 1

    print("{} 클래스별 인스턴스 수:".format(name))
    class_counts = dict(sorted(class_counts.items()))
    for cls, cnt in class_counts.items():
        print(f"Class {cls}: {cnt}개")

count_instance_per_class("train", r"C:\Users\user\Downloads\dataset\dataset\labels\train")
count_instance_per_class("val", r"C:\Users\user\Downloads\dataset\dataset\labels\val")
count_instance_per_class("test", r"C:\Users\user\Downloads\dataset\dataset\labels\test")

train 클래스별 인스턴스 수:
Class 1: 2596개
Class 2: 5132개
Class 3: 663개
val 클래스별 인스턴스 수:
Class 1: 137개
Class 2: 251개
Class 3: 81개
test 클래스별 인스턴스 수:
Class 1: 142개
Class 2: 249개
Class 3: 73개


In [ ]:
# Optuna를 사용하여 다양한 하이퍼파라미터를 시도하며 학습
# 해당 하이파파라미터로 학습을 하며 자동으로 파라미터도 최적화 후 제일 좋은 조합을 출력

def objective(trial):
    lr0 = trial.suggest_float('lr0', 1e-4, 1e-1, log=True)
    batch = trial.suggest_categorical('batch', [8, 16, 32])
    mosaic = trial.suggest_float('mosaic', 0.0, 1.0)
    epochs = 20

    model = YOLO('yolov8n.pt')
    result = model.train(
        data=r"C:\Users\user\Downloads\dataset\dataset\data.yaml",
        epochs=epochs,
        lr0=lr0,
        batch=batch,
        mosaic=mosaic,
        imgsz=640,
        device='cuda'
    )
    
    # 올바른 mAP50-95 반환
    return result.box.map

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
print("Best params:", study.best_params)

[I 2025-05-16 13:42:09,060] A new study created in memory with name: no-name-3f0ac656-f851-41bc-9ee7-9103f7b6a55f


Ultralytics 8.3.0  Python-3.10.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\user\Downloads\dataset\dataset\data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

train: Scanning C:\Users\user\Downloads\dataset\dataset\labels\train... 4560 images, 0 backgrounds, 0 corrupt: 100%|███


train: New cache created: C:\Users\user\Downloads\dataset\dataset\labels\train.cache


val: Scanning C:\Users\user\Downloads\dataset\dataset\labels\val... 426 images, 0 backgrounds, 0 corrupt: 100%|████████

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:01


                   all        426        469      0.855      0.786      0.858      0.593
                  fire        105        137      0.916      0.964      0.976      0.668
              accident        244        251      0.835      0.665      0.778        0.5
              sinkhole         77         81      0.814      0.728      0.821       0.61
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train11


[I 2025-05-16 14:17:27,166] Trial 2 finished with value: 0.5926444300823428 and parameters: {'lr0': 0.0026455694259257664, 'batch': 8, 'mosaic': 0.20555824882899143}. Best is trial 0 with value: 0.6001160435683008.


Ultralytics 8.3.0  Python-3.10.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\user\Downloads\dataset\dataset\data.yaml, epochs=20, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning C:\Users\user\Downloads\dataset\dataset\labels\train.cache... 4560 images, 0 backgrounds, 0 corrupt: 10
val: Scanning C:\Users\user\Downloads\dataset\dataset\labels\val.cache... 426 images, 0 backgrounds, 0 corrupt: 100%|██


In [ ]:
# 최적화 하이퍼파라미터와 파러미터 간 상관관계, 파라미터별 성능 분포 시각화 파트

# 1. 최적화 히스토리(각 trial별 best value 변화)
fig1 = plot_optimization_history(study)
fig1.show()

# 2. 하이퍼파라미터 중요도(어떤 파라미터가 성능에 영향이 큰지)
fig2 = plot_param_importances(study)
fig2.show()

# 3. 파라미터 간 상관관계(Parallel Coordinate)
fig3 = plot_parallel_coordinate(study)
fig3.show()

# 4. 파라미터별 성능 분포(Slice Plot)
fig4 = plot_slice(study)
fig4.show()

In [32]:
# 최적의 하이퍼파라미터로 테스트 진행

def test_model(model):
    # 1. 테스트 데이터셋 설정
    test_params = {
        'data': r"C:\Users\user\Downloads\dataset\dataset\data.yaml",
        'split': 'test',  
        'batch': 8,
        'name': 'disaster_detection_test',
        'plots': True,    
        'save_json': True
    }
    
    # 2. 테스트 실행
    results = model.val(**test_params)
    
    # 3. 결과 출력
    print("\n📊 최종 테스트 결과 리포트")
    print(f"mAP50: {results.box.map50:.4f}")
    print(f"Precision: {np.mean(results.box.p):.4f}")
    print(f"Recall: {np.mean(results.box.r):.4f}")
    print(f"검출된 객체 수: {results.box.nex}")
    
    return results

best_trial_number = study.best_trial.number
best_model_path = f"runs/detect/train{best_trial_number+1}/weights/best.pt"
print("Best model path:", best_model_path)

model = YOLO(best_model_path)
test_result = test_model(model)

Ultralytics 8.3.134  Python-3.10.9 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 179.290.0 MB/s, size: 58.9 KB)


val: Scanning C:\Users\user\Downloads\dataset\dataset\labels\test... 546 images, 0 backgrounds, 0 corrupt: 100%|███████

val: New cache created: C:\Users\user\Downloads\dataset\dataset\labels\test.cache
WARNING Box and segment counts should be equal, but got len(segments) = 207, len(boxes) = 671. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 69/69 [00:02


                   all        546        671      0.577      0.488      0.546      0.289
                 flood        135        207          0          0          0          0
                  fire        106        142      0.885       0.93       0.94      0.508
              accident        236        249      0.697       0.38      0.501      0.242
              sinkhole         69         73      0.726      0.644      0.743      0.408
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving runs\detect\disaster_detection_test\predictions.json...
Results saved to runs\detect\disaster_detection_test

📊 최종 테스트 결과 리포트
mAP50: 0.5461


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [ ]:
# 테스트 이미지 시각화
def visualize_predictions():
    model = YOLO('runs/detect/disaster_detection_v3/weights/best.pt')
    test_images_dir = r"C:\Users\user\Downloads\dataset\dataset\test\images"
    
    # 첫 5개 이미지에 대한 예측
    for img_name in os.listdir(test_images_dir)[:5]:
        img_path = os.path.join(test_images_dir, img_name)
        results = model.predict(
            source=img_path,
            save=True,
            conf=0.5,  # 신뢰도 임계값
            save_dir='test_results'  # 예측 결과 저장 경로
        )
        print(f"{img_name} 예측 완료 → test_results 디렉토리 확인")

# 시각화 실행
visualize_predictions()